In [ ]:
!pip install DeepFace
!pip install keyboard
!pip install xlwings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 9.8 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=cc0d85313515fe84f3c3cfb2475f16d054cd7ee8b8c42b3da671314624fa56af
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.9 MB/s eta 0:00:00


In [ ]:
from deepface import DeepFace
#liveness
import tensorflow as tf
import cv2, numpy, datetime, pygame, keyboard
import openpyxl


In [ ]:
model = 'liveness.model'
model = tf.keras.models.load_model(model)

In [ ]:
# Initialize workbook and worksheet
workbook = openpyxl.load_workbook('attendance.xlsx')
sheet_name = datetime.date.today().isoformat()

try:
    worksheet = workbook[sheet_name]
except KeyError:
    worksheet = workbook.create_sheet(sheet_name)


In [ ]:
# Write headers to the worksheet
worksheet['A1'] = 'STUDENT'
worksheet['B1'] = 'DATE'
worksheet['C1'] = 'TIME'

In [ ]:
students = []
s = 2
i = 0
entered = False
al_entered = False

In [ ]:
cap = cv2.VideoCapture(0)

#pygame

pygame.init()

pg_txt = (39,112,50)
pg_bg = (154,179,157)

font = cv2.FONT_HERSHEY_SIMPLEX
pygame_font = pygame.font.Font('freesansbold.ttf',32)

screen = pygame.display.set_mode((960,540))

background = pygame.image.load('background_image.jpg')
background = pygame.transform.rotozoom(background,0,0.8)
screen.blit(background,(0,0))


#day today
t0 = datetime.date.today()
t0 = t0.day

running = True

while running:

    moment = datetime.datetime.now()
    hour = moment.hour
    minute = moment.minute
    day = moment.day
    month = moment.month
    year = moment.year

    date = f"{day}-{month}-{year}"
    time = f"{hour}:{minute}"

    screen.blit(background,(0,0))

    #check

    if day!=t0:
        t0 = day

        worksheet = workbook.sheets.add(date)

        worksheet.range('A1').value = 'STUDENT'
        worksheet.range('B1').value = 'DATE'
        worksheet.range('C1').value = 'TIME'
        students = []
        s=2
        i = 0
        entered = False
        al_entered = False





    state, frame = cap.read()

    if state!=True:
        break

    res  = DeepFace.find(frame, db_path='./Database/', enforce_detection=False, model_name='VGG-Face')

    if len(res[0]['identity'])>0:

        name = res[0]['identity'][0].split('/')[1].split('\\')[1]
        xmin = int(res[0]['source_x'][0])
        ymin = int(res[0]['source_y'][0])

        w = res[0]['source_w'][0]
        h = res[0]['source_h'][0]

        xmax = int(xmin + w)
        ymax = int(ymin + h)

        #liveness

        img = frame[ymin:ymax, xmin:xmax]
        img = cv2.resize(img, (32,32))
        img = img.astype('float')/255.0
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = numpy.expand_dims(img, axis=0)

        liveness = model.predict(img)

        liveness = liveness[0].argmax()





        if liveness==1:
            cv2.rectangle(frame, (xmin,ymin),(xmax,ymax), (0,255,0),1)
            cv2.rectangle(frame, (xmin, ymin-25),(xmax, ymin),(255,255,255),-1)
            cv2.putText(frame, name, (xmin,ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2,cv2.LINE_AA)

            n_txt = pygame_font.render(name, True, pg_txt, pg_bg)
            n_txtrect = n_txt.get_rect()
            n_txtrect.center =(960-200, 540//2)

            d_txt = pygame_font.render(date, True, pg_txt, pg_bg)
            d_txtrect = d_txt.get_rect()
            d_txtrect.center= (960-200, (540//2)+40)

            t_txt = pygame_font.render(time, True, pg_txt, pg_bg)
            t_txtrect = d_txt.get_rect()
            t_txtrect.center = (960-200, (540//2)+80)


            screen.blit(n_txt, n_txtrect)
            screen.blit(d_txt, d_txtrect)
            screen.blit(t_txt, t_txtrect)

            if keyboard.is_pressed('enter'):

                if name not in students:

                    worksheet.range(f'A{s}').value = name
                    worksheet.range(f'B{s}').value = date
                    worksheet.range(f'c{s}').value = time
                    students.append(name)
                    s+=1
                    entered =True
                else:
                    al_entered = True

            if i<30 and entered==True:
                e_txt = pygame_font.render('entered', True, pg_txt, pg_bg)
                e_txtrect = e_txt.get_rect()
                e_txtrect.center = (960-200, (540//2)+120)
                screen.blit(e_txt, e_txtrect)
                i+=1

            elif i<30 and al_entered==True:
                ae_txt = pygame_font.render('already entered', True, pg_txt, pg_bg)
                ae_txtrect = ae_txt.get_rect()
                ae_txtrect.center = (960-200, (540//2)+120)
                screen.blit(ae_txt, e_txtrect)
                i+=1

            else:
                entered=False
                al_entered = False
                i = 0



    image = pygame.image.frombuffer(frame.tostring(), frame.shape[1::-1], 'BGR')
    image = pygame.transform.rotozoom(image, 0 , 0.8)
    #round corners--
    size = image.get_size()
    rect_img = pygame.Surface(size, pygame.SRCALPHA)
    pygame.draw.rect(rect_img, (255,255,255), (0,0,*size), border_radius=10)

    image = image.copy().convert_alpha()
    image.blit(rect_img, (0,0), None, pygame.BLEND_RGB_MIN)
    #round corner ---

    screen.blit(image, (20,80))


    pygame.display.update()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    #cv2.imshow('attendance', frame)

    #c = cv2.waitKey(1)

    #if c == ord('q'):
        #break

cap.release()
pygame.quit()
#cv2.destroyAllWindows()